In [1]:
from IPython.display import display, HTML
display(HTML("""
<style>
div.container{width:90% !important;}
div.cell.code_cell.rendered{width:100%;}
div.input_prompt{padding:0px;}
div.CodeMirror {font-family:Consolas; font-size:12pt;}
div.text_cell_render.rendered_html{font-size:12pt;}
div.output {font-size:12pt; font-weight:bold;}
div.input {font-family:Consolas; font-size:12pt;}
div.prompt {min-width:70px;}
div#toc-wrapper{padding-top:120px;}
div.text_cell_render ul li{font-size:12pt;padding:5px;}
table.dataframe{font-size:12px;}
</style>
"""))

<b><font size="6" color="red">ch15. DB연동</font></b>
# 1절 SQLite 데이터 베이스 연결
 - SQLite 데이터 베이스는 별도의 DBMS 없이 SQL문 이용해서 DB액세스할 수 있도록 만든 간단한 디스크 기반 DB
 - C라이브러리
 - SQLite는 프로토타입 생성시 사용
 - 프로젝트 : 분석 -> 설계 -> 구현 -> 테스트 -> 고객에게 배포 -> 유지보수 프로토타입(SQLite) 완제품(Oracle, MySQL, Maria, ..)
 - DB browser for SQLite
## 1.1 SQLite browser 설치 및 sqlite 패키지 load

In [3]:
import sqlite3
sqlite3.sqlite_version # 버전

'3.40.1'

In [5]:
import pandas as pd
pd.__version__

'1.5.3'

## 1.2 DB연결
 - (1) DB 연결
 - (2) 객체 생성(cursor)
 - (3) 전송 & 결과(cursor.execute()이용)
 - (4) cursor해제 & DB연결객체 해제

In [6]:
# DB연결 : sqlite로 연결시 DB파일이 없으면 빈 DB파일 생성, 있으면 연결
conn = sqlite3.connect('data/ch15_example.db')
conn

In [7]:
# sql 전송 객체 생성(cursor 객체)
cursor = conn.cursor()
cursor

In [8]:
cursor.execute('''
    CREATE TABLE MEMBER(
        NAME TEXT, 
        AGE  INT,
        EMAIL TEXT
    )
''')

In [ ]:
# cursor.execute('DROP TABLE MEMBER')

In [9]:
cursor.execute('INSERT INTO MEMBER VALUES (\'홍길동\', 20, \'h@h.com\')')
print('수행결과 행수 :', cursor.rowcount)

수행결과 행수 : 1


In [10]:
sql = "INSERT INTO MEMBER VALUES ('김길동', 30, 'K@K.COM')"
cursor.execute(sql)
print('수행 결과 행수 :', cursor.rowcount)
cursor.execute("INSERT INTO MEMBER VALUES ('이길동', 25, 'l@l.com')")
print('수행 결과 행수 :', cursor.rowcount)

수행 결과 행수 : 1
수행 결과 행수 : 1


In [11]:
conn.commit()

In [20]:
# select 전송 결과 : cursor
cursor.execute("select * from member")

In [21]:
# insert, update, delete문 실행 결과 : cursor.rowcount
# select문 실행 결과 : 
        ## fetchall() : 결과를 모두 받을 때(튜플 list)
        ## fetchone() : 결과를 한행씩 받을 때(튜플)
        ## fetchmany(n) : 결과를 n행 받을 때(튜플 list)
members = cursor.fetchall()
members

[('홍길동', 20, 'h@h.com'), ('김길동', 30, 'K@K.COM'), ('이길동', 25, 'l@l.com')]

In [25]:
for member in members:
    print(member)

('홍길동', 20, 'h@h.com')
('김길동', 30, 'K@K.COM')
('이길동', 25, 'l@l.com')


In [26]:
cursor.execute("SELECT * FROM MEMBER ORDER BY AGE")
members = []
while True:
    member = cursor.fetchone()
    if member is None:
        print('데이터 끝')
        break
    members.append(member)

데이터 끝


In [27]:
members

[('홍길동', 20, 'h@h.com'), ('이길동', 25, 'l@l.com'), ('김길동', 30, 'K@K.COM')]

In [28]:
# 최상위 n행 읽기
cursor.execute("SELECT * FROM MEMBER ORDER BY AGE")
for member in cursor.fetchmany(2):
    print(member)

('홍길동', 20, 'h@h.com')
('이길동', 25, 'l@l.com')


In [29]:
cursor.description

(('NAME', None, None, None, None, None, None),
 ('AGE', None, None, None, None, None, None),
 ('EMAIL', None, None, None, None, None, None))

In [30]:
class Member:
    'Member 테이블의 내용을 받을 객체 타입'
    def __init__(self, name, age, email):
        self.name = name
        self.age  = age
        self.email = email
    def __str__(self):
        return "{}\t{}\t{}".format(self.name, self.age, self.email)
    
def to_member(*row): # 튜플매개변수를 받아 Member형 객체를 return
    return Member(row[0], row[1], row[2])

In [31]:
member = to_member('홍길동', 20, 'h@h.com')
print(member)

홍길동	20	h@h.com


In [34]:
cursor.execute('SELECT * FROM MEMBER')
member_list = [] # sql문 수행한 결과를 담을 객체 list
members = cursor.fetchall() # 튜플 list
# print(members)
for member in members:
    member_list.append(to_member(*member))

In [35]:
for member in member_list:
    print(member)

홍길동	20	h@h.com
김길동	30	K@K.COM
이길동	25	l@l.com


In [36]:
# (4) 연결 해제
cursor.close()
conn.close()

## 1.3 SQL 구문에 파라미터 사용하기
 - qmark(DB에 따라 불가한 경우 있음)
 - named(추천)

In [37]:
conn = sqlite3.connect('data/ch15_example.db')
cursor = conn.cursor()
cursor.execute("SELECT * FROM MEMBER WHERE NAME IN ('홍길동','김길동')")
cursor.fetchall()

[('홍길동', 20, 'h@h.com'), ('김길동', 30, 'K@K.COM')]

In [38]:
# 파라미터 사용하기 : qmark 방법 
name1 = input('검색할 이름1?')
name2 = input('검색할 이름2?')
# cursor.execute("SELECT * FROM MEMBER WHERE NAME IN ('"+name1+"','"+name2+"')")
# cursor.execute(f"SELECT * FROM MEMBER WHERE NAME IN ('{name1}','{name2}')")
cursor.execute("SELECT * FROM MEMBER WHERE NAME IN (?, ?)", (name1, name2))
cursor.fetchall()

검색할 이름1?홍길동
검색할 이름2?김길동


[('홍길동', 20, 'h@h.com'), ('김길동', 30, 'K@K.COM')]

In [39]:
# 파라미터 사용하기 : named 방법 
name1 = input('검색할 이름1?')
name2 = input('검색할 이름2?')
cursor.execute("SELECT * FROM MEMBER WHERE NAME IN (:name1, :name2)", {'name1':name1, 
                                                                       'name2':name2})
cursor.fetchall()

검색할 이름1?홍길동
검색할 이름2?김길동


[('홍길동', 20, 'h@h.com'), ('김길동', 30, 'K@K.COM')]

In [ ]:
# Member 테이블에 입력 (사용자로부터 이름, 나이, 메일을 받아 insert)
try :
    name = input('입력할 이름 :')
    age  = int(input('입력할 나이 :'))
except:
    print('유효하지 않은 나이를 입력한 경우, 18세로 초기화')
    age = 18
finally:
    email = input('입력할 메일은 :')
inputdata = {'name':name, 'age':age, 'email':email} # named방식에서 사용
inputdata2 = (name, age, email) # qmark방식에서 사용
sql = "INSERT INTO MEMBER VALUES (:name, :age, :email)"
cursor.execute(sql, inputdata)
conn.commit()
if cursor.rowcount:
    print('저장완료')

In [ ]:
# Member 테이블에 입력 (사용자로부터 이름, 나이, 메일을 받아 insert)
try :
    name = input('입력할 이름 :')
    age  = int(input('입력할 나이 :'))
except:
    print('유효하지 않은 나이를 입력한 경우, 18세로 초기화')
    age = 18
finally:
    email = input('입력할 메일은 :')
newMember = Member(name, age, email)
# print(newMember)
# print(newMember.__dict__)
sql = "INSERT INTO MEMBER VALUES (:name, :age, :email)"
cursor.execute(sql, newMember.__dict__)
conn.commit()
if cursor.rowcount==1:
    print('입력 성공')

In [ ]:
while True:
    try :
        name = input('입력할 이름(종료는 0) :')
        if name=='0':
            break;
        age  = int(input('입력할 나이 :'))
    except:
        print('유효하지 않은 나이를 입력한 경우, 18세로 초기화')
        age = 18
    email = input('입력할 메일은 :')
    newMember = Member(name, age, email)
    # print(newMember)
    # print(newMember.__dict__)
    sql = "INSERT INTO MEMBER VALUES (:name, :age, :email)"
    cursor.execute(sql, newMember.__dict__)
    if cursor.rowcount==1:
        print('입력 성공')
conn.commit()

In [ ]:
cursor.close()
conn.close()

# 2절 오라클 데이터 베이스 연결
 - pip install cx_oracle (oracle 11g까지)
 - pip install oracledb (oracle 12부터)

In [ ]:
import cx_Oracle
cx_Oracle.__version__

In [ ]:
# conn 얻어오는 방법1
conn = cx_Oracle.connect("scott", "tiger", "localhost:1521/xe")
cursor = conn.cursor()
sql = 'SELECT EMPNO "NO", ENAME, JOB, MGR, HIREDATE, SAL, COMM, DEPTNO FROM EMP'
cursor.execute(sql)
emp = cursor.fetchall()
print(emp)

In [40]:
# conn 얻어오는 방법2
import oracledb
oracledb.init_oracle_client()
# conn = oracledb.connect("scott/tiger@localhost:1521/xe")
conn = oracledb.connect(
    user="scott",
    password="tiger",
    host="localhost",
    port=1521,
    sid="xe"
)
cursor = conn.cursor()
sql = 'SELECT EMPNO "NO", ENAME, JOB, MGR, HIREDATE, SAL, COMM, DEPTNO FROM EMP'
cursor.execute(sql)
emp = cursor.fetchall()
print(emp)

[(7369, 'SMITH', 'CLERK', 7902, datetime.datetime(1980, 12, 17, 0, 0), 800.0, None, 20), (7499, 'ALLEN', 'SALESMAN', 7698, datetime.datetime(1981, 2, 20, 0, 0), 1600.0, 300.0, 30), (7521, 'WARD', 'SALESMAN', 7698, datetime.datetime(1981, 2, 22, 0, 0), 1250.0, 500.0, 30), (7566, 'JONES', 'MANAGER', 7839, datetime.datetime(1981, 4, 2, 0, 0), 2975.0, None, 20), (7654, 'MARTIN', 'SALESMAN', 7698, datetime.datetime(1981, 9, 28, 0, 0), 1250.0, 1400.0, 30), (7698, 'BLAKE', 'MANAGER', 7839, datetime.datetime(1981, 5, 1, 0, 0), 2850.0, None, 30), (7782, 'CLARK', 'MANAGER', 7839, datetime.datetime(1981, 6, 9, 0, 0), 2450.0, None, 10), (7788, 'SCOTT', 'ANALYST', 7566, datetime.datetime(1982, 12, 9, 0, 0), 3000.0, None, 20), (7839, 'KING', 'PRESIDENT', None, datetime.datetime(1981, 11, 17, 0, 0), 5000.0, None, 10), (7844, 'TURNER', 'SALESMAN', 7698, datetime.datetime(1981, 9, 8, 0, 0), 1500.0, 0.0, 30), (7876, 'ADAMS', 'CLERK', 7788, datetime.datetime(1983, 1, 12, 0, 0), 1100.0, None, 20), (7900, 

In [42]:
import pandas as pd
emp_df = pd.DataFrame(emp)
emp_df

,0,1,2,3,4,5,6,7
0,7369,SMITH,CLERK,7902.0,1980-12-17,800.0,NaN,20
1,7499,ALLEN,SALESMAN,7698.0,1981-02-20,1600.0,300.0,30
2,7521,WARD,SALESMAN,7698.0,1981-02-22,1250.0,500.0,30
3,7566,JONES,MANAGER,7839.0,1981-04-02,2975.0,NaN,20
4,7654,MARTIN,SALESMAN,7698.0,1981-09-28,1250.0,1400.0,30
5,7698,BLAKE,MANAGER,7839.0,1981-05-01,2850.0,NaN,30
6,7782,CLARK,MANAGER,7839.0,1981-06-09,2450.0,NaN,10
7,7788,SCOTT,ANALYST,7566.0,1982-12-09,3000.0,NaN,20
8,7839,KING,PRESIDENT,NaN,1981-11-17,5000.0,NaN,10
9,7844,TURNER,SALESMAN,7698.0,1981-09-08,1500.0,0.0,30


In [43]:
# select문을 수행한 필드 정보 (NO, ENAME, JOB, ~)
cursor.description

[('NO', <DbType DB_TYPE_NUMBER>, 5, None, 4, 0, False),
 ('ENAME', <DbType DB_TYPE_VARCHAR>, 10, 10, None, None, True),
 ('JOB', <DbType DB_TYPE_VARCHAR>, 9, 9, None, None, True),
 ('MGR', <DbType DB_TYPE_NUMBER>, 5, None, 4, 0, True),
 ('HIREDATE', <DbType DB_TYPE_DATE>, 23, None, None, None, True),
 ('SAL', <DbType DB_TYPE_NUMBER>, 11, None, 7, 2, True),
 ('COMM', <DbType DB_TYPE_NUMBER>, 11, None, 7, 2, True),
 ('DEPTNO', <DbType DB_TYPE_NUMBER>, 3, None, 2, 0, True)]

In [44]:
[desc[0] for desc in cursor.description]

['NO', 'ENAME', 'JOB', 'MGR', 'HIREDATE', 'SAL', 'COMM', 'DEPTNO']

In [45]:
emp_df.columns = [desc[0] for desc in cursor.description]
emp_df.head()

,NO,ENAME,JOB,MGR,HIREDATE,SAL,COMM,DEPTNO
0,7369,SMITH,CLERK,7902.0,1980-12-17,800.0,NaN,20
1,7499,ALLEN,SALESMAN,7698.0,1981-02-20,1600.0,300.0,30
2,7521,WARD,SALESMAN,7698.0,1981-02-22,1250.0,500.0,30
3,7566,JONES,MANAGER,7839.0,1981-04-02,2975.0,NaN,20
4,7654,MARTIN,SALESMAN,7698.0,1981-09-28,1250.0,1400.0,30


In [51]:
# 검색할 이름을 사용자에게 받아 해당 내용을 출력
ename = input('검색할 이름은?').upper()
# print(ename)
sql = "SELECT * FROM EMP WHERE ENAME = :ename"
cursor.execute(sql, {'ename':ename})
emp = cursor.fetchall()
if emp:
    print(emp[0])
else:
    print('입력하신 이름의 데이터는 없습니다')

검색할 이름은?smith
(7369, 'SMITH', 'CLERK', 7902, datetime.datetime(1980, 12, 17, 0, 0), 800.0, None, 20)


In [52]:
fieldnames = [descrip[0] for descrip in cursor.description]
fieldnames

['EMPNO', 'ENAME', 'JOB', 'MGR', 'HIREDATE', 'SAL', 'COMM', 'DEPTNO']

In [53]:
for fieldname, data in zip(fieldnames, emp[0]):
    print("{}:{}".format(fieldname, data))

EMPNO:7369
ENAME:SMITH
JOB:CLERK
MGR:7902
HIREDATE:1980-12-17 00:00:00
SAL:800.0
COMM:None
DEPTNO:20


In [54]:
cursor.close
conn.close

<bound method Connection.close of <oracledb.Connection to scott@(DESCRIPTION=(ADDRESS=(PROTOCOL=tcp)(HOST=localhost)(PORT=1521))(CONNECT_DATA=(SID=xe)))>>

# 3절 MySQL 데이터베이스 연결
 - pip install mysql-connector-python (공식connector)
 - pip install PyMySQL (경량)

In [55]:
%pip install PyMySQL

     ---------------------------------------- 45.3/45.3 kB 2.3 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [57]:
import pymysql

In [1]:
conn = pymysql.connect(
    host = "localhost",
    user="root",
    password="1234",
    db="devdb",
    charset="utf8mb4"
)
cursor = conn.cursor()
sql = "SELECT * FROM PERSONAL"
cursor.execute(sql)
personal = cursor.fetchall()
personal

NameError: name 'pymysql' is not defined

# 4절 연습문제
## 실습형
 - 회원가입 | 전체조회 | 이름찾기 | 메일삭제 | csv보내기 | 종료

In [5]:
def load_cursor():
    global conn
    import oracledb
    oracledb.init_oracle_client()
    conn = oracledb.connect("scott/tiger@localhost:1521/xe")
load_cursor()

 - 1. 입력

In [6]:
def fn1_insert_member_info():
    '입력받은 데이터를 member테이블에 입력'
    cursor = conn.cursor()
    name = input('이름 :')
    tel  = input('전화 :')
    email = input('이메일 :')
    try:
        age = int(input('나이 :'))
        if age>200:
            age = 200
        elif age<0:
            age = 0
    except:
        print('유효하지 않은 나이 입력시 나이는 0으로 초기화')
        age = 0
    try:
        grade = int(input('등급 :'))
        if grade<1:
            grade = 1
        elif grade>5:
            grade = 5
    except:
        print('유효하지 않은 등급 입력시 등급은 1로 초기화')
        grade = 1
    etc = input('기타정보 :')
    sql = "INSERT INTO MEMBER VALUES (:name, :tel, :email, :age, :grade, :etc)"
    inputdata = {'name':name, 'tel':tel, 'email':email, 'age':age, 'grade':grade, 'etc':etc}
    cursor.execute(sql, inputdata)
    conn.commit()
    if cursor.rowcount==1:
        print(name, '님 가입 완료')
    cursor.close()

이름 :홍길동
전화 :010-1212-9090
이메일 :hong@naver.com
나이 :30
등급 :3
기타정보 :신출귀몰
홍길동 님 가입 완료


In [7]:
fn1_insert_member_info()

이름 :김길동
전화 :01-9876-0901
이메일 :kim@naver.com
나이 :54
등급 :4
기타정보 :old
김길동 님 가입 완료


In [15]:
import pandas as pd
cursor = conn.cursor()
sql = "select * from MEMBER order by Name"
cursor.execute(sql)
member = cursor.fetchall()
print(cursor.rowcount)
df = pd.DataFrame(member)
df.columns = [desc[0] for desc in cursor.description]
display(df)
df

2


,NAME,TEL,EMAIL,AGE,GRADE,ETC
0,김길동,01-9876-0901,kim@naver.com,54,4,old
1,홍길동,010-1212-9090,hong@naver.com,30,3,신출귀몰


,NAME,TEL,EMAIL,AGE,GRADE,ETC
0,김길동,01-9876-0901,kim@naver.com,54,4,old
1,홍길동,010-1212-9090,hong@naver.com,30,3,신출귀몰


 - 2. 전체조회

In [ ]:
def fn2_print_members():
    'member 테이블의 내용을 데이터프레임으로 display'
    import pandas as pd
    cursor = conn.cursor()
    sql = "SELECT * FROM MEMBER ORDER BY NAME"
    cursor.execute(sql)
    fieldnames = [description[0] for description in cursor.description]
    df = pd.DataFrame(cursor.fetchall(), columns=fieldnames)
    display(df)
    cursor.close()
fn2_print_members()

- 3. 이름조회

In [ ]:
def fn3_search_member():
    '찾고자하는 이름을 input으로 받아 해당 내용을 데이터프레임으로 display'
    import pandas as pd
    cursor = conn.cursor()
    name = input('찾고자 하는 이름은 ?')
    sql = "SELECT * FROM MEMBER WHERE NAME=:name"
    cursor.execute(sql, {'name':name})
    members = cursor.fetchall()
    fieldnames = [description[0] for description in cursor.description]
    if members:
        df = pd.DataFrame(members, columns=fieldnames)
        display(df)
    else:
        print('해당 이름의 회원이 없습니다')
    cursor.close()
fn3_search_member()

 - 4. 메일 삭제

In [ ]:
def fn4_delete_member():
    '삭제하고자 하는 메일을 input으로 받아 해당 행을 삭제'
    cursor = conn.cursor()
    email = input('삭제하고자 하는 메일은?')
    sql = "DELETE FROM MEMBER WHERE EMAIL=:email"
    cursor.execute(sql, {'email':email})
    if cursor.rowcount:
        print(f'해당 메일({email})을 삭제하였습니다')
    else:
        print('해당 메일의 회원이 없습니다')
    cursor.close()
fn4_delete_member()

- 5. csv 내보내기

In [ ]:
def fn5_save_csv():
    'member 테이블 내용을 csv로 내보내기'
    import pandas as pd
    cursor = conn.cursor()
    sql = "SELECT * FROM MEMBER ORDER BY NAME"
    cursor.execute(sql)
    fieldnames = [description[0] for description in cursor.description]
    df = pd.DataFrame(cursor.fetchall(), columns=fieldnames)
    df.to_csv('data/ch15_member.csv', index=False)
    cursor.close()
fn5_save_csv()

In [ ]:
load_cursor()
while True:
    menu = input("1.회원입력 | 2.전체조회 | 3.이름찾기 | 4.메일삭제 | 5.CSV내보내기 | 0.종료")
    if menu=='1':
        fn1_insert_member_info()
    elif menu=='2':
        fn2_print_members()
    elif menu=='3':
        fn3_search_member()
    elif menu=='4':
        fn4_delete_member()
    elif menu=='5':
        fn5_save_csv()
    elif menu=='0':
        conn.close()
        break
    else:
        print('유효한 메뉴 번호를 입력해 주세요')